# 1.- Instalación y configuración de PySpark (1 punto)
# • Configura correctamente el entorno de PySpark y crea una SparkSession con el nombre AnalisisTelefonos.

In [1]:
!pip install pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, when, corr, round

spark = SparkSession.builder.appName("Analisis Telefonos").getOrCreate()

# 2.- Carga de datos (1 punto)
# • Carga el archivo CSV proporcionado (con el nombre telefonos_sudamerica.csv) en un DataFrame de PySpark. Asegúrate de que el archivo contenga los encabezados.


In [4]:
# Cargar el archivo CSV en un DataFrame de Spark
df = spark.read.csv("telefonos_sudamerica.csv", header=True, inferSchema=True)

+---------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|     pais|       modelo|   marca|precio|memoria_ram|capacidad_bateria|edad_comprador|fecha_venta|
+---------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|Argentina|   Galaxy S21| Samsung|  1000|          8|             4000|            30| 2023-01-15|
|   Brasil|    iPhone 13|   Apple|  1200|          6|             3500|            25| 2023-02-10|
|    Chile| Xiaomi Mi 11|  Xiaomi|   800|          6|             4500|            28| 2023-03-05|
|   Brasil|   Galaxy A72| Samsung|   700|          6|             5000|            22| 2023-02-12|
|     Peru|Redmi Note 10|  Xiaomi|   350|          4|             4000|            34| 2023-01-20|
| Colombia|    iPhone 12|   Apple|   950|          6|             3500|            27| 2023-02-18|
|   Brasil|Motorola Edge|Motorola|   750|          8|             5000|            31| 2023-01-25|
|Argentina

# 3.- Exploración inicial de los datos (1 punto)
# • Muestra las primeras 10 filas del DataFrame y realiza una inspección básica de los tipos de datos de cada columna. ¿Existen valores nulos o erróneos en alguna columna?


In [5]:
# Mostrar las primeras 10 filas del DataFrame
print("Primeras 10 filas:")
df.show(10)

# Realizar una inspección de los tipos de datos y buscar valores nulos
print("Esquema y tipos de datos:")
df.printSchema()

# Contar valores nulos por columna
print("Conteo de valores nulos:")
for columna in df.columns:
    count_nulls = df.filter(col(columna).isNull()).count()
    if count_nulls > 0:
        print(f"Columna '{columna}': {count_nulls} valores nulos.")
    else:
        print(f"Columna '{columna}': 0 valores nulos.")

Primeras 10 filas:
+---------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|     pais|       modelo|   marca|precio|memoria_ram|capacidad_bateria|edad_comprador|fecha_venta|
+---------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|Argentina|   Galaxy S21| Samsung|  1000|          8|             4000|            30| 2023-01-15|
|   Brasil|    iPhone 13|   Apple|  1200|          6|             3500|            25| 2023-02-10|
|    Chile| Xiaomi Mi 11|  Xiaomi|   800|          6|             4500|            28| 2023-03-05|
|   Brasil|   Galaxy A72| Samsung|   700|          6|             5000|            22| 2023-02-12|
|     Peru|Redmi Note 10|  Xiaomi|   350|          4|             4000|            34| 2023-01-20|
| Colombia|    iPhone 12|   Apple|   950|          6|             3500|            27| 2023-02-18|
|   Brasil|Motorola Edge|Motorola|   750|          8|             5000|            31| 202

# 4.- Filtrado de datos (2 puntos)
# • Filtra el DataFrame para obtener únicamente los teléfonos vendidos en Brasil.
# • Luego, filtra esos datos para obtener solo los teléfonos de la marca Samsung

In [6]:
# Filtrar teléfonos vendidos en Brasil
df_brasil = df.filter(df.pais == "Brasil")
print("Teléfonos vendidos en Brasil:")
df_brasil.show()

# Filtrar los teléfonos de la marca Samsung dentro del DataFrame de Brasil
df_samsung_brasil = df_brasil.filter(df_brasil.marca == "Samsung")
print("Teléfonos de la marca Samsung vendidos en Brasil:")
df_samsung_brasil.show()

Teléfonos vendidos en Brasil:
+------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|  pais|       modelo|   marca|precio|memoria_ram|capacidad_bateria|edad_comprador|fecha_venta|
+------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|Brasil|    iPhone 13|   Apple|  1200|          6|             3500|            25| 2023-02-10|
|Brasil|   Galaxy A72| Samsung|   700|          6|             5000|            22| 2023-02-12|
|Brasil|Motorola Edge|Motorola|   750|          8|             5000|            31| 2023-01-25|
+------+-------------+--------+------+-----------+-----------------+--------------+-----------+

Teléfonos de la marca Samsung vendidos en Brasil:
+------+----------+-------+------+-----------+-----------------+--------------+-----------+
|  pais|    modelo|  marca|precio|memoria_ram|capacidad_bateria|edad_comprador|fecha_venta|
+------+----------+-------+------+-----------+-----------------

# 5.- Operaciones de agrupamiento y agregación (2 puntos)
# • Agrupa los datos por país y calcula la venta promedio de los teléfonos (promedio de precio).
# • Agrupa los datos por marca y calcula el número de teléfonos vendidos por cada marca.

In [7]:
# Agrupar por país y calcular el precio promedio
df_promedio_pais = df.groupBy("pais").agg(round(avg("precio"), 2).alias("precio_promedio"))
print("Venta promedio de teléfonos por país:")
df_promedio_pais.show()

# Agrupar por marca y calcular el número de teléfonos vendidos
df_conteo_marca = df.groupBy("marca").agg(count("*").alias("numero_de_ventas"))
print("Número de teléfonos vendidos por marca:")
df_conteo_marca.show()

Venta promedio de teléfonos por país:
+---------+---------------+
|     pais|precio_promedio|
+---------+---------------+
|Argentina|          750.0|
|     Peru|          725.0|
|    Chile|          600.0|
|   Brasil|         883.33|
| Colombia|          950.0|
+---------+---------------+

Número de teléfonos vendidos por marca:
+--------+----------------+
|   marca|numero_de_ventas|
+--------+----------------+
|Motorola|               1|
|  Xiaomi|               3|
| Samsung|               4|
|   Apple|               2|
+--------+----------------+



# 6.- Análisis por rango de edad (1 punto)
# • Crea una nueva columna en el DataFrame que agrupe a los compradores por rango de edad (por ejemplo, 18-25 años, 26-35 años, 36-50 años, 51+ años).
# • Agrupa los datos por este nuevo rango de edad y muestra el promedio de precio de los teléfonos vendidos para cada rango de edad.


In [8]:
# Crear la nueva columna 'rango_edad'
df_edad = df.withColumn(
    "rango_edad",
    when((col("edad_comprador") >= 18) & (col("edad_comprador") <= 25), "18-25 años")
    .when((col("edad_comprador") >= 26) & (col("edad_comprador") <= 35), "26-35 años")
    .when((col("edad_comprador") >= 36) & (col("edad_comprador") <= 50), "36-50 años")
    .otherwise("51+ años")
)

# Agrupar por el nuevo rango de edad y mostrar el precio promedio
df_promedio_edad = df_edad.groupBy("rango_edad").agg(round(avg("precio"), 2).alias("precio_promedio"))
print("Precio promedio de teléfonos por rango de edad:")
df_promedio_edad.show()

Precio promedio de teléfonos por rango de edad:
+----------+---------------+
|rango_edad|precio_promedio|
+----------+---------------+
|26-35 años|         764.29|
|18-25 años|          800.0|
+----------+---------------+



# 7.- Análisis de correlación (1 punto)
# • Calcula la correlación entre las columnas memoria_ram y precio. ¿Qué tipo de correlación existe entre estas dos variables?

In [9]:
# Calcular la correlación entre memoria_ram y precio
correlation = df.stat.corr("memoria_ram", "precio")
print(f"La correlación entre memoria_ram y precio es: {correlation}")

# Conclusión
if correlation > 0.5:
    print("Existe una fuerte correlación positiva. A mayor memoria RAM, mayor es el precio.")
elif correlation < -0.5:
    print("Existe una fuerte correlación negativa. A mayor memoria RAM, menor es el precio.")
else:
    print("Existe una correlación débil o nula entre las variables.")

La correlación entre memoria_ram y precio es: 0.6301449500096707
Existe una fuerte correlación positiva. A mayor memoria RAM, mayor es el precio.


# 8.- Filtrado por características del teléfono (1 punto)
# • Filtra los teléfonos que tengan una memoria RAM mayor a 6 GB y una batería mayor a 4000 mAh. ¿Cuántos teléfonos cumplen con esta condición?


In [10]:
# Filtrar teléfonos que cumplen con la condición
df_filtrado_caracteristicas = df.filter(
    (col("memoria_ram") > 6) & (col("capacidad_bateria") > 4000)
)

# Contar cuántos teléfonos cumplen con la condición
count_telefonos = df_filtrado_caracteristicas.count()
print(f"Número de teléfonos que cumplen con memoria RAM mayor a 6 GB y bateria mayor a 4000 [mAh]: {count_telefonos}")

# Mostrar el resultado
df_filtrado_caracteristicas.show()

Número de teléfonos que cumplen con memoria RAM mayor a 6 GB y bateria mayor a 4000 [mAh]: 2
+------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|  pais|       modelo|   marca|precio|memoria_ram|capacidad_bateria|edad_comprador|fecha_venta|
+------+-------------+--------+------+-----------+-----------------+--------------+-----------+
|Brasil|Motorola Edge|Motorola|   750|          8|             5000|            31| 2023-01-25|
|  Peru|   Galaxy S21| Samsung|  1100|          8|             4500|            33| 2023-02-22|
+------+-------------+--------+------+-----------+-----------------+--------------+-----------+



# 9.- Guardar los resultados (1 punto)
# • Guarda el DataFrame resultante de los filtros y agregaciones anteriores en un nuevo archivo CSV llamado resultados_analisis.csv.


In [11]:
# Guardar el DataFrame filtrado en un nuevo archivo CSV
# Nota: Spark crea un directorio con múltiples archivos.
df_filtrado_caracteristicas.write.csv("resultados_analisis.csv", header=True, mode="overwrite")

print("DataFrame guardado exitosamente en el directorio 'resultados_analisis.csv'.")

DataFrame guardado exitosamente en el directorio 'resultados_analisis.csv'.
